In [1]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")

In [2]:
import torch

In [3]:
from nnetworks.layers.config import LstmConfig, DenseInput, MaskedConv1DConfig, ResBlockConfig
from nnetworks.models.config import MuNuSepLstmConfig, MuNuSepResNetConfig
from nnetworks.models.munusep_lstm import MuNuSepLstm
from nnetworks.models.munusep_resnet import MuNuSepResNet

from nnetworks.models.config_manager import save_model_cfg, model_from_yaml

# LSTM based model

In [17]:
cfg = MuNuSepLstmConfig(
        lstm_layers=[
            LstmConfig(
                5,
                64,
                1,
                return_sequences=True,
                dropout = 0.,
                bidirectional=True,
                norm = {"MaskedLayerNorm12": None}
            )
        ],
        pooling_type="Average",
        dense_layers=[
            DenseInput(
                (64)*2, 
                32,
                activation = {'LeakyReLU': dict(negative_slope=0.1)},
                dropout = 0.1,
                do_norm = True
                ),
            
            DenseInput(
                32, 
                2,
                activation = {'Softmax': {"dim": 1}},
                dropout = 0.,
                do_norm = False
                ),
        ]
    )

model = MuNuSepLstm(cfg)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

# test
model(torch.rand(2,10,5))

40866


tensor([[0.7491, 0.2509],
        [0.2636, 0.7364]], grad_fn=<SoftmaxBackward0>)

In [18]:
name_for_model_arch = "munusep_all_rnn"
save_model_cfg(cfg, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

In [19]:
model_from_yaml(MuNuSepLstm, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

MuNuSepLstm(
  (lstm_layers): ModuleList(
    (0): LstmLayer(
      (lstm_layer): LSTM(5, 64, batch_first=True, bidirectional=True)
      (norm_layer): MaskedLayerNorm12()
    )
  )
  (pooling): GlobalAveragePooling1DMasked()
  (dense_layers): ModuleList(
    (0): DenseBlock(
      (dropout_layer): Dropout(p=0.1, inplace=False)
      (dense_layer): Linear(in_features=128, out_features=32, bias=True)
      (activation): LeakyReLU(negative_slope=0.1)
      (norm1d): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): DenseBlock(
      (dropout_layer): Dropout(p=0.0, inplace=False)
      (dense_layer): Linear(in_features=32, out_features=2, bias=True)
      (activation): Softmax(dim=1)
    )
  )
)

# ResNet Based model

In [7]:
# Tiny ResNet
cfg = MuNuSepResNetConfig(
        res_blocks = [
            ResBlockConfig(
                id = MaskedConv1DConfig(
                        in_channels = 5,
                        out_channels = 64,
                        kernel_size = 3,
                        strides = 1,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                    ),
                cd = MaskedConv1DConfig(
                        in_channels = 64,
                        out_channels = 128,
                        kernel_size = 3,
                        strides = 2,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                    ),
                skip = MaskedConv1DConfig(
                        in_channels = 5,
                        out_channels = 128,
                        kernel_size = 5,
                        strides = 2,
                        activation = {"LeakyReLU": {"negative_slope": 0.1}},
                        dropout = 0.1,
                        norm = {"LayerNorm": None}
                )
            )
        ],
        pooling_type = "Average",
        dense_layers = [
            DenseInput(
                in_features = 256, 
                units = 32,
                activation = {'ReLU': None},
                dropout = 0.1,
                do_norm = True
                ),
            DenseInput(
                in_features = 32, 
                units = 2,
                activation = {'Softmax': {"dim": 1}},
                do_norm = False,
                dropout = 0.
                ),
        ]
    )

model = MuNuSepResNet(cfg)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

# test
model(torch.rand((3,10, 5)), torch.ones((3,10,1)))

38050


tensor([[0.4476, 0.5524],
        [0.7519, 0.2481],
        [0.3390, 0.6610]], grad_fn=<SoftmaxBackward0>)

In [8]:
name_for_model_arch = "munusep_all_resnet"
save_model_cfg(cfg, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

In [9]:
model_from_yaml(MuNuSepResNet, f"/home/albert/Baikal-ML/nnetworks/models/configurations/{name_for_model_arch}.yaml")

MuNuSepResNet(
  (res_blocks): ModuleList(
    (0): ResBlock(
      (conv_id): MaskedConv1D(
        (conv): Conv1d(5, 64, kernel_size=(3,), stride=(1,))
        (dropout): Dropout(p=0.1, inplace=False)
        (activation): LeakyReLU(negative_slope=0.1)
        (norm_layer): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (conv_cd): MaskedConv1D(
        (conv): Conv1d(64, 128, kernel_size=(3,), stride=(2,))
        (dropout): Dropout(p=0.1, inplace=False)
        (activation): LeakyReLU(negative_slope=0.1)
        (norm_layer): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (conv_skip): MaskedConv1D(
        (conv): Conv1d(5, 128, kernel_size=(5,), stride=(2,))
        (dropout): Dropout(p=0.1, inplace=False)
        (activation): LeakyReLU(negative_slope=0.1)
        (norm_layer): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (pooling): GlobalAveragePooling1DMasked()
  (dense_layers): ModuleList(
    (0): DenseBlock

In [20]:
import numpy as np

In [24]:
np.arange(10)[(torch.ones(10)==1).numpy()]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])